In [3]:
#using https://bioimagesuiteweb.github.io/webapp/mni2tal.html
import pandas as pd
import numpy as np
import importlib
import util.Data
import util.Plotly as plots

In [4]:
importlib.reload(plots)

<module 'util.Plotly' from 'D:\\scul\\uaic-ub\\licenta\\code\\util\\Plotly.py'>

# Data

In [5]:
csv_filename = r'data/final_coordinates-no_conversion.csv'

In [7]:
data_df = util.Data.import_data(csv_filename)
data_df.shape

(1719, 19)

In [5]:
#split data by mni and talairach type coordinates
split_mask = data_df['keywords'].str.contains('MNI', regex=True)
df_mni = data_df[split_mask]
df_tal = data_df[~split_mask]

In [6]:
df_mni_coord =df_mni[['X(R)','Y(A)','Z(S)']]
df_tal_coord =df_tal[['X(R)','Y(A)','Z(S)']]

In [7]:
df_mni_coord.shape,df_tal_coord.shape

((367, 3), (289, 3))

In [8]:
mni_path = r'data/mni.csv'
df_mni_coord.to_csv(mni_path,index = False,header=False)
tal_path = r'data/tal.csv'
df_tal_coord.to_csv(tal_path,index = False,header=False)

In [9]:
csv_mni_conv_filename = r'data/mni_converted.csv'
csv_tal_conv_filename = r'data/tal_converted.csv'


In [10]:
data_mni_conv_df = pd.read_csv(csv_mni_conv_filename)
data_tal_conv_df = pd.read_csv(csv_tal_conv_filename)

In [48]:
#MNI
df_mni.reset_index(drop=True, inplace=True)
data_mni = pd.merge(df_mni, data_mni_conv_df, left_index=True, right_index=True)#pd.concat([df_mni, data_mni_conv_df], axis=1, ignore_index=True)
assert data_mni.shape[0] == df_mni.shape[0]

check_list1 = ['X(R)','Y(A)','Z(S)']
check_list2 = ['MNIX','MNIY','MNIZ']
for ch1,ch2 in zip(check_list1,check_list2):
    assert data_mni[ch1].equals(data_mni[ch2])

In [49]:
#Talairach
df_tal.reset_index(drop=True,inplace=True)
data_tal = pd.merge(df_tal,data_tal_conv_df,left_index=True,right_index=True)
assert data_tal.shape[0]==df_tal.shape[0]

check_list1 = ['X(R)','Y(A)','Z(S)']
check_list2 = ['TALX','TALY','TALZ']
for ch1,ch2 in zip(check_list1,check_list2):
    assert data_tal[ch1].equals(data_tal[ch2])

In [93]:
#Concatenate the two dataframe to create the final one
data_final = pd.concat([data_mni, data_tal], axis=0)

In [94]:
# Split BA name and BA number
data_final[['BA_name','BA_no']] = data_final['BA'].str.rsplit("(", 1, expand=True)
data_final['BA_no'] = data_final['BA_no'].str.replace(')','')

C:\Users\Chh\AppData\Local\Temp\ipykernel_35288\3746375327.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_final['BA_no'] = data_final['BA_no'].str.replace(')','')


In [95]:
# BA number column
#String
# data_final['BA_no'] = data_final['BA_no'].astype(str)
# data_final['BA_no'] = data_final['BA_no'].str.replace('None','nan')
#Float
data_final['BA_no']= pd.to_numeric(data_final['BA_no'],downcast='integer')

In [97]:
data_final.dtypes

importance        object
author            object
title             object
table_name        object
keywords          object
Left/Right        object
Name              object
Broadman Area     object
t                float64
X(R)               int64
Y(A)               int64
Z(S)               int64
z-score          float64
NV               float64
p value          float64
cluster size     float64
index_col          int64
MNIX               int64
MNIY               int64
MNIZ               int64
BA                object
TALX               int64
TALY               int64
TALZ               int64
BA_name           object
BA_no            float64
dtype: object

In [98]:
csv_final_path = r'data/final_coordinates-conversion-v2.csv'
data_final.to_csv(csv_final_path,index = False,header=True)
